In [1]:
# ThoroughBet Simulation


## Load necessary modules

In [1]:
import numpy as np

from utils import timestamp, YEAR
from utils.arrayview import ArrayView, TimeseriesView

from prediction.models.fit_model import TSModel
from prediction.models.model_parameters import ModelParameters
from prediction.models.factor_management import FactorList
from prediction.models.preprocessing import load_slices

from prediction.models.prediction import factornames_trimmed
from prediction.models.parameters import factor_build_end

## Load data

In [3]:
av = ArrayView.from_file('../datadev/brain_final2cut.av.bcolz')

In [32]:
tsav = load_slices(path='../datadev/')

## Preprocessing

In [4]:
pars = ModelParameters(av, oos_start=factor_build_end+YEAR, depth=3, lmbd=10, verbose=True)

In [5]:
fl = FactorList(av, factornames_trimmed)

In [6]:
fl.preprocess(pars)
factors = fl.asmatrix()

INFO:models:Getting factors from av and rescaling...


. . . . .

INFO:models:Filling in missing values...
INFO:models:Computing each factor as linear combination of all the others...


 . . . . .

INFO:models:Number of missing patterns: 7754


 . . . . . . .


## Fit conditional logit model

In [7]:
tsmod = TSModel(factors, tsav, pars)

In [8]:
tsmod.fit_slices()

INFO:models:Fitting slice 0
INFO:models:Fitting slice 1
INFO:models:Fitting slice 2
INFO:models:Fitting slice 3
INFO:models:Fitting slice 4
INFO:models:Fitting slice 5
INFO:models:Fitting slice 6
INFO:models:Fitting slice 7
INFO:models:Fitting slice 8
INFO:models:Fitting slice 9


In [9]:
tsmod.write_simdata('simdata.p')

In [4]:
av.columns

NameError: name 'av' is not defined

In [6]:
av.fields

['event_id',
 'run_id',
 'runner_id',
 'start_time',
 'course',
 'distance',
 'going',
 'obstacle',
 'race_class',
 'win_time',
 'prize',
 'date_of_birth',
 'sex',
 'dam',
 'sire',
 'jockey',
 'trainer',
 'weight',
 'draw',
 'result',
 'beaten_length',
 'handicap',
 'equipment',
 'bsp',
 'isp',
 'bfid',
 'bdid',
 'morning_est',
 'morning_prob',
 'thb_est',
 'napstats_tipssum']

In [8]:
av.keys()

['z7e1e4a30a',
 'z5e3e2bf8e',
 'z2f01df1e4',
 'z1a1aff9a5',
 'runner_id',
 'course',
 'z7603edce1',
 'zef5a850eb',
 'speed',
 'zba15a816a',
 'z98f7dab30',
 'sp_back_volume_final',
 'zf8b32c847',
 'za97562293',
 'run_id',
 'z8ca2ea7f2',
 'sp_back_volume',
 'date_of_birth',
 'win_time',
 'zc64f78aa9',
 'z76228ad9c',
 'dam',
 'start_time',
 'zbba3f4349',
 'z12c6cabe4',
 'z5fa2eaf85',
 'z873364109',
 'race_class',
 'za5c443236',
 'prize',
 'z1121cb627',
 'z698cc55af',
 'zd81bcd220',
 'zb87666d6d',
 'z6809c316d',
 'z8d2862a46',
 'zf47a136cc',
 'z8f2529f11',
 'weight',
 'z09c1cfca1',
 'za31432965',
 'draw',
 'sex',
 'zb5dc14260',
 'zb85d3de89',
 'result',
 'z3d11336f4',
 'zab7c330ee',
 'zac38414de',
 'za5cf91a5d',
 'zec0c22a48',
 'z14536acf4',
 'zd35b970d2',
 'z34b78e584',
 'z999f82560',
 'z215129aeb',
 'z6f11029f7',
 'jockey',
 'z2a69f7bab',
 'z412893062',
 'zc31456152',
 'z50810021e',
 'bfid',
 'z1b0bd4349',
 'sp_lay_volume_final',
 'z027f9f0f5',
 'z3bc9c1546',
 'isp',
 'zf0e03d75d',
 'nor

In [9]:
av.event_id[:100]

array([178463, 178463, 178463, 178463, 178463, 178463, 178463, 178463,
       178463, 178463, 178463, 178463, 178462, 178462, 178462, 178462,
       178462, 178462, 178462, 178462, 178462, 178462, 178467, 178467,
       178467, 178467, 178467, 178467, 178467, 178467, 178467, 178467,
       178467, 178467, 178467, 178464, 178464, 178464, 178464, 178464,
       178464, 178464, 178464, 178464, 178464, 178464, 178465, 178465,
       178465, 178465, 178465, 178465, 178465, 178465, 178465, 178465,
       178465, 178465, 178466, 178466, 178466, 178466, 178466, 178466,
       178469, 178469, 178469, 178469, 178469, 178469, 178469, 178469,
       178469, 178468, 178468, 178468, 178468, 178468, 178468, 178468,
       178468, 178468, 178468, 178468, 178470, 178470, 178470, 178470,
       178470, 178470, 178470, 178470, 178470, 178470, 178470, 178470,
       178474, 178474, 178474, 178474])

In [10]:
av. runner_id

array([234306, 234307, 234308, ..., 133652, 446997, 307291])

In [12]:
av.shape()

AttributeError: shape

In [13]:
av.attributes

AttributeError: attributes

In [14]:
timestamp(av.start_time[0])

<Timestamp 2013-01-01 00:57:00.000000>

In [15]:
fl.preprocess

NameError: name 'fl' is not defined